# 爬虫基础: Requests + BeautifulSoup

“爬虫”，是访问互联网网页 ->定位网页元素 -> 爬取网页内容的过程。<br>
在实际工作中，访问的工作交给urllib或者requests完成；爬取的工作则交给xpath, BeatifulSoup乃至正则合作完成。<br>
想学习爬虫知识，首先需要了解HTTP基础请求方法：get和post

## HTTP方法：GET和POST

### GET方法

<font color='red'>请注意，查询字符串（名称/值对）是在 GET 请求的 URL 中发送的</font><br>
如：```https://www.baidu.com/s?ie=utf-8&wd=python```

有关 GET 请求的其他一些注释：

* GET 请求可被缓存
* GET 请求保留在浏览器历史记录中
* GET 请求可被收藏为书签
* GET 请求不应在处理敏感数据时使用
* GET 请求有长度限制
* GET 请求只应当用于取回数据

### POST方法

<font color='red'>请注意，查询字符串（名称/值对）是在 POST 请求的 HTTP 消息主体中发送的</font><br>
```
POST http://dcms-ml-dev.dc68032.easn.morningstar.com/automation/api/namesimilar HTTP/1.1
Host: dcms-ml-dev.dc68032.easn.morningstar.com
Connection: keep-alive
Content-Length: 103
accept: application/json
Origin: http://dcms-ml-dev.dc68032.easn.morningstar.com
User-Agent: Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36
Content-Type: application/json
Referer: http://dcms-ml-dev.dc68032.easn.morningstar.com/apidocs/
Accept-Encoding: gzip, deflate
Accept-Language: en-US,en;q=0.9,zh-CN;q=0.8,zh;q=0.7
{
  "country": "",
  "onlyactive": "0",
  "querytype": "share",
  "text": "S&P 500",
  "universe": ""
}
```

有关 POST 请求的其他一些注释：

* POST 请求不会被缓存
* POST 请求不会保留在浏览器历史记录中
* POST 不能被收藏为书签
* POST 请求对数据长度没有要求

<img src="./image/2018-09-17 12_15_17-HTTP 方法GET对比POST.png" width="100%">

## Requests

requests是一个很实用的Python HTTP客户端库，编写爬虫和测试服务器响应数据时经常会用到。可以说，Requests 完全满足如今网络的需求。<br>
Requests的官方文档 http://docs.python-requests.org/en/master/ <br>
安装方式一般采用```pip install requests``` <br>

### GET请求

In [2]:
import requests
r = requests.get('https://www.baidu.com')
print(r)

<Response [200]>


#### 带参数的Get请求
这里需要加入headers，给一个user-agent伪装一下自己

In [3]:
parameters = {'ie': 'utf-8', 'wd':'看病'}
headers = {
"User-Agent": "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36"}
requestforbing = requests.get('https://www.baidu.com/s', 
                              params=parameters, 
                              headers=headers)
# requestforbing = requests.get('https://www.baidu.com/s', params=parameters)
# print('Response: {0}, Whole URL: {1}, Text: {2}'.format(requestforbing, requestforbing.url, requestforbing.content))
print('Response: {0}, Whole URL: {1}'.format(requestforbing, requestforbing.url))

Response: <Response [200]>, Whole URL: https://www.baidu.com/s?ie=utf-8&wd=%E7%9C%8B%E7%97%85


#### 将请求的网页内容保存到本地

In [6]:
rstream = requests.get('https://www.baidu.com/s', 
                       params=parameters, 
                       headers=headers, 
                       stream=True)
print(rstream)
try:
    with open('./html/baiduresult.html', 'wb') as fd:
        for chunk in rstream.iter_content(1000):
            fd.write(chunk)
except Exception as e:
    print(e)

<Response [200]>


[baiduresult.html](./html/baiduresult.html)

### POST请求

大部分的业务应用，其实都是POST请求。<br>
现在POST请求的参数一般通过JSON字符串组成，返回的结果往往也是JSON字符串<br>
以下是具体的例子。

以Form的形式发送请求

In [7]:
import requests
import json
url = 'http://httpbin.org/post'
d = {'key1': 'value1', 'key2': 'value2'}
r = requests.post(url, data=d)
print(r.text)

{
  "args": {}, 
  "data": "", 
  "files": {}, 
  "form": {
    "key1": "value1", 
    "key2": "value2"
  }, 
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate", 
    "Content-Length": "23", 
    "Content-Type": "application/x-www-form-urlencoded", 
    "Host": "httpbin.org", 
    "User-Agent": "python-requests/2.20.0", 
    "X-Bluecoat-Via": "fc8ec97cbb42049b"
  }, 
  "json": null, 
  "origin": "183.62.146.29, 183.62.146.29", 
  "url": "https://httpbin.org/post"
}



以Json的形式发送请求

In [8]:
url = 'http://httpbin.org/post'
s = json.dumps({'key1': 'value1', 'key2': 'value2'})
r = requests.post(url, data=s)
print(r.text)

{
  "args": {}, 
  "data": "{\"key1\": \"value1\", \"key2\": \"value2\"}", 
  "files": {}, 
  "form": {}, 
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate", 
    "Content-Length": "36", 
    "Host": "httpbin.org", 
    "User-Agent": "python-requests/2.20.0", 
    "X-Bluecoat-Via": "fc8ec97cbb42049b"
  }, 
  "json": {
    "key1": "value1", 
    "key2": "value2"
  }, 
  "origin": "183.62.146.29, 183.62.146.29", 
  "url": "https://httpbin.org/post"
}



## BeautifulSoup

简单来说，Beautiful Soup是python的一个库，最主要的功能是从网页抓取数据。官方解释如下：<br>
```
Beautiful Soup提供一些简单的、python式的函数用来处理导航、搜索、修改分析树等功能。它是一个工具箱，通过解析文档为用户提供需要抓取的数据，因为简单，所以不需要多少代码就可以写出一个完整的应用程序。

Beautiful Soup自动将输入文档转换为Unicode编码，输出文档转换为utf-8编码。你不需要考虑编码方式，除非文档没有指定一个编码方式，这时，Beautiful Soup就不能自动识别编码方式了。然后，你仅仅需要说明一下原始编码方式就可以了。

Beautiful Soup已成为和lxml、html5lib一样出色的python解释器，为用户灵活地提供不同的解析策略或强劲的速度。
```


### 安装

```
pip install beautifulsoup4
pip install lxml
pip install html5lib
```

Beautiful Soup支持Python标准库中的HTML解析器,还支持一些第三方的解析器，如果我们不安装它，则 Python 会使用 Python默认的解析器，lxml 解析器更加强大，速度更快，推荐安装。<br>
<font color='red'>lxml处理具有多个pre标签的html的时候，只能解析第一个pre标签，此时建议还是通过BeautifulSoup(markup, "html.parser")进行解析网页</font>

<img src="./image/2018-09-17 20_05_34-Python爬虫利器二之Beautiful Soup的用法.png" width="100%">

特别详细的内容，可以参考官方文档，下面就开始正式举例说明了~~~

[官方文档](https://beautifulsoup.readthedocs.io/zh_CN/latest/)

之前应星宇所托，写了个爬取山东地区汽油柴油价格行情的爬虫示例。

<img src='./image/2018-09-17 20_11_54.png' width='100%'/>

点击第一个链接，可以进入明细页面:

<img src='./image/2018-09-17 20_14_16.png' width='100%'/>

如何实现呢？我们一步一步来~~~

In [9]:
# 导入所需要的python包
import re
import requests
from pyquery import PyQuery as pq
from bs4 import  BeautifulSoup
import pandas as pd
import os
from IPython.display import display, HTML

设置url，header，搜索文本变量

In [10]:
url = "http://www.baidu.com/s?ie=utf-8&wd={0}&tn=monline_4_dg"
headers = {
    "User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_3) AppleWebKit/600.5.17 (KHTML, like Gecko) Version/8.0.5 Safari/600.5.17"
}
searchtext = "2018年9月17日 山东地区汽油柴油价格行情 东方财富"
url = url.format(searchtext)

通过requests完成查询，并且通过pyquery获取特定属性下面的链接信息

In [13]:
response = requests.get(url, headers=headers)
response.encoding = 'utf-8'
# print(response.text)
page = pq(response.text)
baiduurls = [(site.attr('href'), site.text().encode('utf-8')) for site in
                page('div.result.c-container  h3.t  a').items()]
print(baiduurls)

[('http://www.baidu.com/link?url=zQboJU2YO411sCt6FT1d2MdXNXflmVRJ0u7oSqDAk1zmHZF-KEd06yrXSRPqt6wy37K0BAPCzEtfPskSLGAHFkzXfmEZVWaURtsIfrpPUGO', b'2018\xe5\xb9\xb49\xe6\x9c\x8817\xe6\x97\xa5\xe5\xb1\xb1\xe4\xb8\x9c\xe5\x9c\xb0\xe5\x8c\xba\xe6\xb1\xbd\xe6\xb2\xb9\xe6\x9f\xb4\xe6\xb2\xb9\xe4\xbb\xb7\xe6\xa0\xbc\xe8\xa1\x8c\xe6\x83\x85 _ \xe4\xb8\x9c\xe6\x96\xb9\xe8\xb4\xa2\xe5\xaf\x8c\xe7\xbd\x91'), ('http://www.baidu.com/link?url=0pFXD-T0SMhYT1LG6d2R_pE5mvaJV3q5t_LZrnG0ZUCKVxqptMTCdFhtYnE02JAVdBJyEjFArqM5kHIvzT53XEw_2OyViTuiRrV_Bg2dqta', b'2018\xe5\xb9\xb49\xe6\x9c\x8817\xe6\x97\xa5\xe5\x8d\x8e\xe4\xb8\x9c\xe5\x9c\xb0\xe5\x8c\xba\xe6\xb1\xbd\xe6\xb2\xb9\xe6\x9f\xb4\xe6\xb2\xb9\xe4\xbb\xb7\xe6\xa0\xbc\xe8\xa1\x8c\xe6\x83\x85 _ \xe4\xb8\x9c\xe6\x96\xb9\xe8\xb4\xa2\xe5\xaf\x8c\xe7\xbd\x91'), ('http://www.baidu.com/link?url=brrbXcVc7O-sWg_obWya7dBcRJOkVPj-8LiiRqjiYUvMvimQgC78VDyMrPdqo6yQXl4LmajSuXYxx1vVSFcslY-niAUqzJ7cOHhCkxVqteW', b'2018\xe5\xb9\xb47\xe6\x9c\x8817\xe6\x97\xa5\xe9\x99\x95\xe8

因为百度做了url转义，所以需要再用requests做一次get操作，从而获取真正需要获取的链接

In [17]:
originalURLs = []
for tmpurl in baiduurls:
    tmpPage = requests.get(tmpurl[0], allow_redirects=False)
#     print(tmpPage.text, tmpPage.headers)
    if tmpPage.status_code == 200:
#         print('200')
        urlMatch = re.search(r'URL=\'(.*?)\'', tmpPage.text.encode('utf-8'), re.S)
        originalURLs.append((urlMatch.group(1), tmpurl[1].decode("utf-8")))
    elif tmpPage.status_code == 302:
#         print('302')
        originalURLs.append((tmpPage.headers.get('location'), tmpurl[1].decode("utf-8")))
    else:
        print('No URL found!!')
print(originalURLs)

[('http://finance.eastmoney.com/news/1356,20180917946941610.html', '2018年9月17日山东地区汽油柴油价格行情 _ 东方财富网'), ('http://finance.eastmoney.com/news/1356,20180917946941397.html', '2018年9月17日华东地区汽油柴油价格行情 _ 东方财富网'), ('http://futures.eastmoney.com/news/1514,20180717908141283.html', '2018年7月17日陕西地区汽油柴油价格行情 _ 东方财富网'), ('http://futures.eastmoney.com/news/1514,20180912944475986.html', '2018年9月12日陕西地区汽油柴油价格行情 _ 东方财富网'), ('http://www.sohu.com/a/42268295_119536', '2015年11月17日山东地区汽油柴油价格行情'), ('http://data.eastmoney.com/cjsj/oil_default.html', '全国油价数据走势一览 _ 数据中心 _ 东方财富网'), ('http://appcert.eastmoney.com/info/detail/201903091064569266', '3月9日唐山地区汽油柴油价格持稳 中国财经门户,提供专业的财经、...'), ('http://forex.eastmoney.com/a/201903131068429498.html', '3月13日山东地区汽油柴油价格行情 _ 东方财富网'), ('http://forex.eastmoney.com/a/201902181046357466.html', '2018年2月18日河北地区汽油柴油价格行情 _ 东方财富网'), ('http://www.sohu.com/a/61671920_119536', '2016年3月3日陕西地区汽油柴油价格行情-新闻频道-手机搜狐')]


既然真正的url已经都拿到了，那么我们就可以进入具体的网页一探究竟

In [18]:
searcharray = searchtext.split()
searchdate = ''
findurl = ''
print(searcharray)
for url in originalURLs:
#     print(url[1], url[0])
    if len(searcharray) >=2 \
            and searcharray[0] in url[1] \
            and searcharray[1] in url[1] \
            and ('futures.eastmoney' in url[0] or 
                 'finance.eastmoney' in url[0]):
        print("东方财富网获得信息：{0}, 网址：{1}".format(url[1], url[0]))
        searchdate = searchtext.split()[0]
        findurl = url[0]
        break
#             parsegasinfo(searchdate, url[0])

['2018年9月17日', '山东地区汽油柴油价格行情', '东方财富']
东方财富网获得信息：2018年9月17日山东地区汽油柴油价格行情 _ 东方财富网, 网址：http://finance.eastmoney.com/news/1356,20180917946941610.html


既然拿到最终需要去爬取的URL了，那么就需要BeautifulSoup去拿文本了

In [19]:
fullgastextinfo = requests.get(findurl)
fullgastextinfo.encoding = 'utf-8'
# 这里使用lxml作为BS4的解析引擎
soup = BeautifulSoup(fullgastextinfo.text, features='lxml')
print(soup.head.text, soup.title.text)
sourcetxtfile = './output/sourcetxt/source_{0}.txt'.format(searchdate)
# 将爬取的信息去空白行之后，存入本地文件
with open(sourcetxtfile, 'w', encoding='utf-8') as f:
    for index, line in enumerate(soup.text.split('\n')):
        if line.split():
#             print("write to file")
#             print("count:", index + 1, "content:", line)
            f.write(line + '\n')





2018年9月17日山东地区汽油柴油价格行情 _ 东方财富网









        function isMobile() {
            var ua = navigator.userAgent;
            var res = false;
            var ipad = ua.match(/(iPad).*OS\s([\d_]+)/),
                isIphone = !ipad && ua.match(/(iPhone\sOS)\s([\d_]+)/),
                isAndroid = ua.match(/(Android)\s+([\d.]+)/),
                isMobile = isIphone || isAndroid;
            if (isMobile) {
                res = true;
            } else {
                res = false;
            }
            return res;
        }
        if (isMobile()) {
            location.href = "https://emwap.eastmoney.com/info/detail/20180917946941610";
        }
    





 2018年9月17日山东地区汽油柴油价格行情 _ 东方财富网


根据之前的知识，查看具体油价信息的地址

<img src='./image/2018-09-17 20_47_51.png' width='100%' />

获得XPATH示例：
```
//*[@id="ContentBody"]/p[1]
//*[@id="ContentBody"]/p[2]
...
```
既然确定文本一定在非常规矩的段落中，那就可以干活了

既然知道文本在div中，且id为ContentBody，那么就可以直接定位div部分

接着，我们再去找段落，用刚刚的结果，再通过find_all('p')就搞定了

In [20]:
# help(soup.find_all)
print(soup.find('div', id='ContentBody').find_all('p'))

[<p>　　山东东营华星石化今日<span id="Info.353"><a class="infokey " href="http://data.eastmoney.com/cjsj/yjtz/default.html" target="_blank">成品油</a></span>报价：国五92#汽油8850元/吨，密度0.72；国五95#汽油无报价，密度0.72；国六92#汽油(高效)8900元/吨，新出；国五0#普柴7800元/吨，密度0.84；国五0#车柴7900元/吨；国六0#车柴7900元/吨；催柴无报价，停报。</p>, <p>　　山东东营正和集团今日成品油报价：国四90#汽油无货，密度0.73；国四93#汽油无报价，密度0.739；国四97#汽油无报价，密度0.748；国六92#汽油8850元/吨，密度0.74；国六95#汽油9000元/吨，密度0.74；国五0#普通柴油7830元/吨。燃料油13#7830元/吨；国五0#车柴7930元/吨；国六0#车柴7930元/吨。</p>, <p>　　山东东营神驰炼厂今日成品油报价：国六89#组分汽油8700元/吨，密度0.728；国六92#汽油8800元/吨，密度0.737；国六95#汽油8900元/吨，密度0.740；普通0#柴油无报价，密度0.823；国六-0#车柴无报价，密度0.813；国六高标92#汽油无报价，密度0.73；国六高标95#汽油无报价，密度0.73；0#普柴无报价，密度0.823；国六0#车柴无报价，密度0.8139；清洁柴油0#7600。</p>, <p class="res-edit">
                (责任编辑：DF120)
            </p>]


我们接着再往下来~~~

注意singlegasprovider.text的用法，其能够去除段落中的所有html标签。

In [35]:
for singlegasprovider in soup.find('div', id='ContentBody').find_all('p'):
    print(singlegasprovider.text)

　　山东东营华星石化今日成品油报价：国五92#汽油8850元/吨，密度0.72；国五95#汽油无报价，密度0.72；国六92#汽油(高效)8900元/吨，新出；国五0#普柴7800元/吨，密度0.84；国五0#车柴7900元/吨；国六0#车柴7900元/吨；催柴无报价，停报。
　　山东东营正和集团今日成品油报价：国四90#汽油无货，密度0.73；国四93#汽油无报价，密度0.739；国四97#汽油无报价，密度0.748；国六92#汽油8850元/吨，密度0.74；国六95#汽油9000元/吨，密度0.74；国五0#普通柴油7830元/吨。燃料油13#7830元/吨；国五0#车柴7930元/吨；国六0#车柴7930元/吨。
　　山东东营神驰炼厂今日成品油报价：国六89#组分汽油8700元/吨，密度0.728；国六92#汽油8800元/吨，密度0.737；国六95#汽油8900元/吨，密度0.740；普通0#柴油无报价，密度0.823；国六-0#车柴无报价，密度0.813；国六高标92#汽油无报价，密度0.73；国六高标95#汽油无报价，密度0.73；0#普柴无报价，密度0.823；国六0#车柴无报价，密度0.8139；清洁柴油0#7600。

                (责任编辑：DF120)
            


讲真，我是期望把数据表格化的，目前来看有点乱，所以还需要加入正则方面的内容。<br>
然后pandas就出场了，这里有些大材小用，但是用的顺手就选它了~~~

In [21]:
import re
temp = r'wwqq；油无报rwww。，：ioopp；。。ee'
temp = re.sub('(\W)', ' ', temp)
temp = re.sub('( ){2,}', ' ', temp)
print(temp)

wwqq 油无报rwww ioopp ee


In [49]:
gasinfo = r'国四93#汽油无报价'
pattern = re.compile(r'(.*)(无报价|无货)')
gastype = pattern.search(gasinfo).group(1)
print(pattern.search(gasinfo).group(0), gastype)

国四93#汽油无报价 国四93#汽油


然后用正则硬刚：

In [22]:
%%time
dfresult = pd.DataFrame(columns=('releasedate',
                                 'provider',
                                 'producttype',
                                 'productprice'))
dfindex = 0
for singlegasprovider in soup.find('div', id='ContentBody').find_all('p'):
    content = singlegasprovider.text
    if (len(content) > 0 and 
       re.match(r'(.*)今日', content) is not None):
        content = content.replace('。','；')
        provider = re.match(r'(.*)今日', content).group(1)
        pattern = re.compile(r'报价：(.*)')
        gasinfo = pattern.search(content)
        if gasinfo is not None:
            gaslist = gasinfo.group(1).split(r'；')
            for gas in gaslist:
                gastypeprice = gas.split(r'，')
                if ('无报价' in gastypeprice[0] or
                   '无货' in gastypeprice[0]):
                    pattern = re.compile(r'(.*)(无报价|无货)')
                    gastype = pattern.search(gastypeprice[0]).group(1)
                    gasprice = r'无报价'
                else:
                    pattern = re.compile(r'([1-9]\d*元/吨)')
                    if pattern.search(gastypeprice[0]) is None:
                        continue
                    gasprice = pattern.search(gastypeprice[0]).group(1)
                    gastype = gastypeprice[0].replace(gasprice, '')
                dfresult.loc[dfindex] = {'releasedate': searchdate.strip(),
                                         'provider': provider.strip(),
                                         'producttype': gastype.strip(),
                                         'productprice': gasprice}
                dfindex += 1
display(dfresult)

,releasedate,provider,producttype,productprice
0,2018年9月17日,山东东营华星石化,国五92#汽油,8850元/吨
1,2018年9月17日,山东东营华星石化,国五95#汽油,无报价
2,2018年9月17日,山东东营华星石化,国六92#汽油(高效),8900元/吨
3,2018年9月17日,山东东营华星石化,国五0#普柴,7800元/吨
4,2018年9月17日,山东东营华星石化,国五0#车柴,7900元/吨
5,2018年9月17日,山东东营华星石化,国六0#车柴,7900元/吨
6,2018年9月17日,山东东营华星石化,催柴,无报价
7,2018年9月17日,山东东营正和集团,国四90#汽油,无报价
8,2018年9月17日,山东东营正和集团,国四93#汽油,无报价
9,2018年9月17日,山东东营正和集团,国四97#汽油,无报价


Wall time: 210 ms


最后将数据存储到csv中：

In [23]:
dfresult.to_csv('./output/result_{0}.csv'.format(searchdate), encoding='utf_8_sig')